#### Single Source Shortest Path (Dijkstra) Algorithm

Implementation of Dijkstra's algorithm using priority queue (min-heap)

In [3]:
class PriorityQueue:
    def __init__(self, capacity=10, items=[]):
        self.empty = (0,'Null')
        if len(items)>0:
            assert capacity >= len(items), "capacity must be greater than or equal to the number of items!"
            self.N = len(items)
            self.queue = [self.empty] + items + [self.empty] * (capacity-self.N)
            self.positions = {item[1]:i for i,item in enumerate(self.queue) if item != self.empty}
            # heapify
            self.heapify()
        else:    
            self.queue = [self.empty] * (capacity+1) 
            self.N = 0

    def heapify(self):
        # start from the last parent node up to root
        for i in range(self.N//2, 0, -1):
            # apply heapify down
            self.heapify_down(i)

    def heapify_up(self, i):
        if i > 1:
            current_item = self.queue[i]
            parent_item = self.queue[i//2]
            if current_item[0] < parent_item[0]:
                # swap with parent
                self.queue[i//2] = current_item
                self.queue[i] = parent_item
                # update positions of swapped items
                self.positions[self.queue[i//2][1]] = i//2
                self.positions[self.queue[i][1]] = i
                # recurse from parent position
                self.heapify_up(i//2) 

    def heapify_down(self, i):
        if 2*i > self.N:
            return 
        elif 2*i < self.N:    
            # get index of child with the smaller key
            if self.queue[2*i][0] <= self.queue[2*i+1][0]:
                j = 2*i
            else:
                j= 2*i+1    
        else:
            j = 2*i
        current_item = self.queue[i]
        if current_item[0] > self.queue[j][0]:
            # swap with smallest child
            self.queue[i] = self.queue[j]
            self.queue[j] = current_item
            # update positions of swapped items
            self.positions[self.queue[i][1]] = i
            self.positions[self.queue[j][1]] = j
            # recurse from child position
            self.heapify_down(j)

    def insert(self, item):
        # insert item into first unoccupied slot
        self.queue[self.N+1] = item
        self.positions[item[1]] = self.N+1
        # heapify up
        self.heapify_up(self.N+1)        
        self.N += 1

    def extract_min(self):
        # swap root with the last item
        min_item = self.queue[1]
        self.positions[min_item[1]] = -1 # set position of removed item to -1
        self.queue[1] = self.queue[self.N]
        self.queue[self.N] = self.empty
        self.N -= 1
        # heapify down
        self.heapify_down(1)
        return min_item

    def delete(self, i):
        current_item = self.queue[i]
        self.positions[current_item[1]] = -1 # set position of removed item to -1
        # swap with last item
        self.queue[i] = self.queue[self.N]
        self.queue[self.N] = self.empty
        self.N -= 1
        # check if we need to heapify up or down
        if current_item[0] < self.queue[i//2][0]:
            self.heapify_up(i)
        else:
            self.heapify_down(i)
        return current_item

    def update_key(self, item_id, new_key):
        # get current position of item 
        i = self.positions[item_id]
        if i == -1:
            # item not in priority queue
            return
        (old_key, _) = self.queue[i]
        # update the item key
        self.queue[i] = (new_key, item_id)
        # check if we need to heapify up or down
        if new_key < old_key:
            self.heapify_up(i)
        else:
            self.heapify_down(i)

    def __str__(self):
        print(f"N = {self.N}")
        return str(self.queue)  

In [27]:
def dijkstra(adjacency_list, verbose=False):
    # pick source node
    s = 'a'
    # initilize shortest distances
    d = {v:float('inf') for v in adjacency_list.keys()}
    d[s] = 0
    # intialize parents
    p = {v:None for v in adjacency_list.keys()}
    # insert all nodes priority queue
    Q = PriorityQueue(items=[(d_v, v) for v,d_v in d.items()], capacity=len(d))
    print(Q)
    # initiale explored set
    S = []
    # perform dijkstra
    while len(S) < len(d):
        # extract min
        (d_v, v) = Q.extract_min()
        # add v to explored set
        S.append(v)
        # update shortest distances to all nodes w adjacent to v
        for (w, l_vw) in adjacency_list[v]:
            if d[w] > d[v] + l_vw:
                # update with new shortest distance
                d[w] = d[v] + l_vw
                # update parent
                p[w] = v
                Q.update_key(w, d[w])
        if verbose:
            print(f"\nFound shortest path to {v} with distance {d_v}")
            print(f"Updated priority queue: {Q}")        

    # return shortest paths
    shortest_paths = {v:(d[v],[]) for v in adjacency_list.keys()}
    for v in adjacency_list.keys():
        u = v
        while u is not None:
            shortest_paths[v][1].insert(0, u)
            u = p[u]

    return shortest_paths



In [32]:
# example graph (adjacency_list implemented using dictionary for clarity)
adjacency_list = {'a':[('b',3),('d',7)], 'b':[('a',3),('d',2),('c',4)], 'c':[('b',4),('d',5),('e',6)], 'd':[('a',7),('b',2),('c',5),('e',4)], 'e':[('c',6),('d',4)]}

paths = dijkstra(adjacency_list, verbose=True)
print(f"\nShortest paths:")
paths

N = 5
[(0, 'Null'), (0, 'a'), (inf, 'b'), (inf, 'c'), (inf, 'd'), (inf, 'e')]

Found shortest path to a with distance 0
N = 4
Updated priority queue: [(0, 'Null'), (3, 'b'), (7, 'd'), (inf, 'c'), (inf, 'e'), (0, 'Null')]

Found shortest path to b with distance 3
N = 3
Updated priority queue: [(0, 'Null'), (5, 'd'), (inf, 'e'), (7, 'c'), (0, 'Null'), (0, 'Null')]

Found shortest path to d with distance 5
N = 2
Updated priority queue: [(0, 'Null'), (7, 'c'), (9, 'e'), (0, 'Null'), (0, 'Null'), (0, 'Null')]

Found shortest path to c with distance 7
N = 1
Updated priority queue: [(0, 'Null'), (9, 'e'), (0, 'Null'), (0, 'Null'), (0, 'Null'), (0, 'Null')]

Found shortest path to e with distance 9
N = 0
Updated priority queue: [(0, 'Null'), (0, 'Null'), (0, 'Null'), (0, 'Null'), (0, 'Null'), (0, 'Null')]

Shortest paths:


{'a': (0, ['a']),
 'b': (3, ['a', 'b']),
 'c': (7, ['a', 'b', 'c']),
 'd': (5, ['a', 'b', 'd']),
 'e': (9, ['a', 'b', 'd', 'e'])}